### Load ODYM

In [143]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import openpyxl
import pylab

# Specify path to dynamic stock model and to datafile, relative
MainPath = os.path.join('..', 'odym', 'modules')
DataPath = os.path.join('..', 'docs', 'files')
sys.path.insert(0, MainPath)

# add ODYM module directory to system path, absolute
sys.path.insert(0, os.path.join(os.getcwd(), '..', 'odym', 'modules'))
sys.path.insert(0, os.path.join(os.getcwd(), '..', 'docs', 'files'))

# Import ODYM files
import ODYM_Classes as msc  # import the ODYM class file
import ODYM_Functions as msf  # import the ODYM function file
import dynamic_stock_model as dsm  # import the dynamic stock model library




### Define MFA system

In [144]:
# Make list of years
year_start = 2000
year_end = 2010
years = list(np.arange(year_start, year_end))

model_classification = dict()

# Classification for time labelled 'Time' must always be present, with Items containing a list of odered integers representing years, months, or other discrete time intervals
model_classification['Time'] = msc.Classification(Name='Time', Dimension='Time', ID=1, Items=years)

# Classification for elements labelled 'Element' must always be present, with Items containing a list of the symbols of the elements covered.
# Now track only carbon (C)
model_classification['Element'] = msc.Classification(Name='Elements', Dimension='Element', ID=2, Items=['C'])

# Get model time start, end, and duration:
model_time_start = int(min(model_classification['Time'].Items))
model_time_end = int(max(model_classification['Time'].Items))
model_duration = model_time_end - model_time_start

In [145]:
index_table = pd.DataFrame({'Aspect': ['Time', 'Element'],  # 'Time' and 'Element' must be present!
                            'Description': ['Model aspect "time"', 'Model aspect "Element"'],
                            'Dimension': ['Time', 'Element'],  # 'Time' and 'Element' are also dimensions
                            'Classification': [model_classification[Aspect] for Aspect in ['Time', 'Element']],
                            'IndexLetter': ['t',
                                            'e']})  # Unique one letter (upper or lower case) indices to be used later for calculations.

# Default indexing of IndexTable, other indices are produced on the fly
index_table.set_index('Aspect', inplace=True)
#index_table

In [146]:
mfa_system = msc.MFAsystem(Name='Sawnwood system',
                           Geogr_Scope='Europe',
                           Unit='kt',
                           ProcessList=[],
                           FlowDict={},
                           StockDict={},
                           ParameterDict={},
                           Time_Start=model_time_start,
                           Time_End=model_time_end,
                           IndexTable=index_table,
                           Elements=index_table.loc['Element'].Classification.Items)  # Initialize MFA system

In [147]:
# Start with empty process list, only process numbers (IDs) and names are needed.
mfa_system.ProcessList = []
mfa_system.ProcessList.append(msc.Process(Name='Semifinished wood products - outside system boundary', ID=0))
mfa_system.ProcessList.append(msc.Process(Name='Wood product production, coniferous', ID=1))
mfa_system.ProcessList.append(msc.Process(Name='Wood product production, non-coniferous', ID=2))
mfa_system.ProcessList.append(msc.Process(Name='Value added market 1', ID=3))
mfa_system.ProcessList.append(msc.Process(Name='Value added market 2', ID=3))
mfa_system.ProcessList.append(msc.Process(Name='Value added market 3', ID=3))

In [150]:
# Parameters
# Consumption values per year
placeholders = []
for index in range(model_duration):
    new_val = index + 100
    new_val *= index * 1.03
    placeholders.append(new_val)

# Total consumption per year, kt/year
values_total_consumption_per_year = np.array(placeholders)

# Composition of elements in consumption, now track only carbon
# This could be per process composition if needed
composition_total_consumption = np.array([1.0])

# Shares

# Yield_Par_Shredder   = np.array([0.92,0.1,0.92]) # Yield for EoL Machine into scrap: Fe: 92%, Cu: 10%, Mn: 92%.
# Yield_Par_Demolition = np.array([0.95,0.02,0.95]) # Yield for EoL Building into scrap: Fe: 95%, Cu: 2%, Mn: 95%.
# Yield_Par_Remelting  = np.array([0.96,1,0.5]) # Yield for EoL Machine into scrap: Fe: 96%, Cu: 100%, Mn: 50%.

# ParameterDict = {}
#
# # Define parameters for inflow of EoL machines and buildings with indices 'te' (years x element) and matching time series Values (array with size 31 x 3).
# # In a more advanced setup the parameters are defined in a data template and then read into the software.
# ParameterDict['F_0_1']  = msc.Parameter(Name = 'Inflow_Eol_Machines', ID = 1, P_Res = 1,
#                                         MetaData = None, Indices = 'te',
#                                         Values = np.einsum('t,e->te',Values_Par_EoL_Machines,Composition_Par_EoL_Machines),
#                                         Unit = 'kt/yr')
# ParameterDict['F_0_2']  = msc.Parameter(Name = 'Inflow_Eol_Buildings', ID = 2, P_Res = 2,
#                                         MetaData = None, Indices = 'te',
#                                         Values = np.einsum('t,e->te',Values_Par_EoL_Buildings,Composition_Par_EoL_Buildings),
#                                         Unit = 'kt/yr')
#
# # Define parameter yield (gamma) with index 'e' (element) and matching time series Values(array with size 3).
# # In a more advanced setup the parameters are defined in a data template and then read into the software.
# ParameterDict['Gamma_1'] = msc.Parameter(Name = 'Shredding yield',  ID = 3, P_Res = 1,
#                                          MetaData = None, Indices = 'e',
#                                          Values = Yield_Par_Shredder, Unit = '1')
# ParameterDict['Gamma_2'] = msc.Parameter(Name = 'Demolition yield', ID = 4, P_Res = 2,
#                                          MetaData = None, Indices = 'e',
#                                          Values = Yield_Par_Demolition, Unit = '1')
# ParameterDict['Gamma_3'] = msc.Parameter(Name = 'Remelting yield',  ID = 5, P_Res = 3,
#                                          MetaData = None, Indices = 'e',
#                                          Values = Yield_Par_Remelting, Unit = '1')
#
# # Assign parameter dictionary to MFA system:
# Dyn_MFA_System.ParameterDict = ParameterDict